In [ ]:
conda install -c conda-forge spacy

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::pytest-astropy==0.5.0=py37_0
  - defaults/osx-64::pytest-arraydiff==0.3=py37h39e3cac_0
  - defaults/osx-64::anaconda==2019.07=py37_0
  - defaults/osx-64::pytest-doctestplus==0.3.0=py37_0
  - defaults/noarch::conda-verify==3.4.2=py_1
  - defaults/osx-64::conda-build==3.18.8=py37_0
  - defaults/osx-64::pytest-openfiles==0.3.2=py37_0
  - defaults/osx-64::pytest-remotedata==0.3.1=py37_0
  - defaults/osx-64::astropy==3.2.1=py37h1de35cc_0
  - defaults/osx-64::conda-package-handling==1.7.2=py37h22f3db7_0
  - defaults/osx-64::conda==4.7.10=py37_0
  - defaults/noarch::path.py==12.0.1=py_0
  - defaults/osx-64::pytest==5.0.1=py37_0
  - defaults/noarch::pluggy==0.12.0=py_0
failed with current_repodata.json, will retry with next repodata source.
Initial quick solve with frozen env failed.  Unfreezing env and trying again.
Solvin

In [1]:
#Gathering info from a user
print("Design a Quiz")
print("Which book would you like to use?")
print ("Here are your choices:\n")
print("The Best Bad Luck I Ever Had \n The Lions of Little Rock\n The Paper Cowboy\n The Thing I'm Most Afraid Of\n Anne of Green Gables")
book = input()
if book == "The Best Bad Luck I Ever Had":
    narrator = "Dit"
    text = "Book_1"
elif book == "The Lions of Little Rock":
    narrator = "Marlee"
    text = "Book_2"
elif book == "The Paper Cowboy":
    narrator = "Tommy"
    text = "Book_3"
elif book == "The Thing I'm Most Afraid Of":
    narrator = "Becca"
    text = "Book_4"
elif book == "Anne of Green Gables":
    narrator = ''
    text = "Book_5"
else:
    print("I'm sorry. That book is not in our database.")

Design a Quiz
Which book would you like to use?
Here are your choices:

The Best Bad Luck I Ever Had 
 The Lions of Little Rock
 The Paper Cowboy
 The Thing I'm Most Afraid Of
 Anne of Green Gables
Anne of Green Gables


In [4]:
#Creating a file dictionary
#This assumes the files are broken down into chapters in a folder
import os
import nltk
import spacy
import pandas as pd
nlp = spacy.load('en')

#Creating the file dictionary
# point this to the data directory
direc = "/Users/kristinlevine/Desktop/NLP_Project/" + text 
ext = '.txt' # Select your file delimiter

file_dict = {} # Create an empty dict

# Select only files with the ext extension
txt_files = [i for i in os.listdir(direc) if os.path.splitext(i)[1] == ext]

# Iterate over your txt files
for f in txt_files:
    # Open them and assign them to file_dict
    with open(os.path.join(direc,f)) as file_object:
        file_dict[f] = file_object.read()

OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is deprecated as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models. If you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")

In [ ]:
print("There are", len(txt_files), 'chapters in this book.')
print("Which chapter would you like to analyze?")
chapter = input()

In [33]:
#To extract persons and locations
def extract_entities(doc_id, doc_text):
    analyzed_doc = nlp(doc_text)

    doc_persons = {}
    doc_locations = {}
    
    for entity in analyzed_doc.ents:
        if entity.text.strip() != "" and entity.label_ == "PERSON":
            prev_token = analyzed_doc[entity.start-1]
            if prev_token.text in ("Doc", "Dr.", "Mr.", "Mrs.", 'Ms.'):
                name = prev_token.text + ' '+ entity.text.strip()
                if name not in doc_persons.keys():
                    relevant_sentence = (doc_id, entity.sent.text)
                    doc_persons[name] = list()
                if name in doc_persons.keys():
                    relevant_sentence = (doc_id, entity.sent.text)
                    doc_persons[name].append(relevant_sentence)  
            else:
                if entity.text.strip() not in doc_persons.keys():
                    relevant_sentence = (doc_id, entity.sent.text)
                    doc_persons[entity.text.strip()] = list()
                
                if entity.text.strip() in doc_persons.keys():
                    relevant_sentence = (doc_id, entity.sent.text)
                    doc_persons[entity.text.strip()].append(relevant_sentence)  
                
    for entity in analyzed_doc.ents:
        if entity.text.strip() != "" and (entity.label_ == 'LOC' or entity.label_ == "GPE"):
            
            if entity.text.strip() not in doc_locations.keys():
                relevant_sentence = (doc_id, entity.sent.text)
                doc_locations[entity.text.strip()] = list()
                
            if entity.text.strip() in doc_locations.keys():
                relevant_sentence = (doc_id, entity.sent.text)
                doc_locations[entity.text.strip()].append(relevant_sentence)  
            
    return doc_persons, doc_locations

In [34]:
def find_most_popular_entities(entity_dictionary):
    
    list_most_mentions = {}
    
    for entity in entity_dictionary:
        x = []
        for i in range(len(entity_dictionary[entity])):
            x.append(len(entity_dictionary[entity][i]))
        list_most_mentions[entity] = sum(x)
        
    # sort through the entities in the dictionary by the number of sentences
    
    return list_most_mentions

In [35]:
a, b = extract_entities(chapter + '.txt', file_dict[chapter +'.txt'])
tp = find_most_popular_entities(a)
tl = find_most_popular_entities(b)
print(tp)

{'Emma Walker': 2, 'Harry Otis': 2, 'Dit': 6, 'Ollie': 2, 'Raymond': 2, 'Earl': 2, 'Pearl': 2, 'Robert': 2, 'Lois': 2, 'Mrs. Pooley': 10, 'Goods Store': 2, 'Uncle': 6, 'Robert E. Lee': 2, 'Lee': 4, 'Mr. Walker': 2}


In [36]:
#Persons
if len(tp) > 0:
    chapter_persons = pd.DataFrame.from_dict(tp, orient = 'index')
    chapter_persons = chapter_persons.rename(columns = {0:'Count'})
    chapter_persons = chapter_persons.sort_values(by=['Count'], ascending = False)
#print(chapter_persons)

In [37]:
#Locations
if len(tl) > 0:
    chapter_locations = pd.DataFrame.from_dict(tl, orient = 'index')
    chapter_locations = chapter_locations.rename(columns = {0:'Count'})
    chapter_locations = chapter_locations.sort_values(by=['Count'], ascending = False)
else:
    chapter_locations = []
#print(chapter_locations)

In [38]:
combined_persons = {}
combined_locations = {}

for item in txt_files: 
    persons, locations = extract_entities(item, file_dict[item])

#For Persons
    for per in persons.keys():
        if per not in combined_persons.keys():
            combined_persons[per] = list()
        
        if per in combined_persons.keys():
            combined_persons[per].append(persons.get(per))

#For Locations
    for loc in locations.keys():
        if loc not in combined_locations.keys():
            combined_locations[loc] = list()
        
        if loc in combined_locations.keys():
            combined_locations[loc].append(locations.get(loc))

In [39]:
#Creating a List of the top characters in the book
all_characters = find_most_popular_entities(combined_persons)
char_in_book = pd.DataFrame.from_dict(all_characters, orient = 'index')
char_in_book = char_in_book.rename(columns = {0:"Count"})
char_in_book = char_in_book.sort_values(by=['Count'], ascending = False)
main_char = char_in_book.head(30)
people = main_char.index.tolist()
#We also need to remove the narrator from this list because he/she can't talk to him/herself.
words_remove = ['Moundville', "Negra"]
if narrator in people:
    people.remove(narrator)
else: 
    pass

for m in words_remove:
    if m in people:
        people.remove(m)
    else:
        pass

print(people)

['Emma', 'Dr. Griffith', 'Elbert', 'Mrs. Walker', 'Bobby', 'Doc', 'Mr. Walker', 'Pearl', 'Doc Haley', 'Mrs. Pooley', 'Raymond', 'Mrs. Seay', 'Chip', 'Mama', 'Davidson', 'Earl', 'Mary', 'Uncle', 'Robert', 'Jim', 'Lois', 'Jim Dang', 'Jim Dang-It', 'Della', 'Ulman', 'Elman', 'Mr. Fulton']


In [40]:
#Creating a List of the top 20 locations in the book
all_locations = find_most_popular_entities(combined_locations)
loc_in_book = pd.DataFrame.from_dict(all_locations, orient = 'index')
loc_in_book = loc_in_book.rename(columns = {0:"Count"})
loc_in_book = loc_in_book.sort_values(by=['Count'], ascending = False)
main_loc = loc_in_book.head(20)
location = main_loc.index.tolist()
words_to_remove = ["ain’t", 'Lover', "Cordelia", '☺', "A.M.", "Curtis", "Negro", "Red", "a.k.a", "Busia"]

for i in people:
    if i in location:
        location.remove(i)
    elif narrator in location:
        location.remove(narrator)
    else:
        pass

for j in words_to_remove:
    if j in location:
        location.remove(j)
    else: 
        pass
        
print(location)

['Moundville', 'Selma', 'Boston', 'Alabama', 'the Black Warrior', 'South', 'Chicago', 'Negras', 'Treasure Island', 'Kentucky', 'Massachusetts', 'New York', 'Haley', 'Albany', 'the United States', 'Germany']


In [41]:
#Check the characters/locations in each chapters against the main_character/locations list.
#This makes sure we are asking about a fairly major character/location; not someone/someplace who is mentioned only once.

chapter_characters = []
for i in range(len(chapter_persons)):
    for x in people:
        if chapter_persons.index[i] == x:
            chapter_characters.append(chapter_persons.index[i])
        else:
            pass    
    
chapter_loc = []
for i in range(len(chapter_locations)):
    for x in location:
        if chapter_locations.index[i] == x:
            chapter_loc.append(chapter_locations.index[i])
        else:
            pass    

In [42]:
print(chapter_characters)
print(chapter_loc)

['Mrs. Pooley', 'Uncle', 'Raymond', 'Earl', 'Pearl', 'Robert', 'Lois', 'Mr. Walker']
['Moundville', 'Alabama', 'Selma']


In [29]:
print(book)
print("Chapter", chapter, ": Questions for Discussion")


#This accounts for if the book has a narrator or not:
if narrator == "":
    narrator = chapter_characters[0]
    i = 1
else:
    i = 0

#This will print up to three character questions per chapter
if len(chapter_characters) == 0:
    pass
elif len(chapter_characters) == 1:
    print("What did", narrator, "talk about with", chapter_characters[i], "?")
elif len(chapter_characters) == 2:
    print("What did", narrator, "talk about with", chapter_characters[i], "?")
    print("What did", narrator, "and", chapter_characters[i+1], "discuss?")
else:
    print("What did", narrator, "talk about with", chapter_characters[i], "?")
    print("What did", narrator, "and", chapter_characters[i+1], "discuss?")
    print("What did", narrator, "say to", chapter_characters[i+2], "?")
    
#This will print a location question:
if len(chapter_loc) == 0:
    pass
else:
    print("Why is", chapter_loc[0], "mentioned in this chapter?")
    
# We'd print these questions, any vocab words, etc, to a pdf that the user could then print out.

The Thing I'm Most Afraid Of
Chapter 1 : Questions for Discussion
What did Becca talk about with Dr. Teresa ?
What did Becca and Chrissy discuss?
Why is Austria mentioned in this chapter?


In [15]:
# Need to figure out how to deal with characters with prefixes "Mrs. Walker" vs. "Walker"
#We'd write these questions to a pdf